In [1]:
import os.path
%load_ext autoreload
%autoreload 2
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
from toolsets.API_gets import inchi_to_smiles, smiles_to_xlogp
from pubchempy import Compound, get_compounds

In [2]:
master_dir = '/Users/fanzhoukong/Documents/GitHub/Libgen_data/polyphenols'
df = pd.read_csv(os.path.join(master_dir,  'polyphenols-libgen-input.csv'))

In [3]:
from rdkit import Chem
from toolsets.std_list_prep import check_mol

In [8]:
smiles = []
inchikey = []
for index, row in tqdm(df.iterrows(), total= len(df)):
    if row['SMILES']!= row['SMILES'] or check_mol(row['SMILES']) is np.NAN:
    # break
        cc = get_compounds(row['Common name'], 'name')
        if len(cc)>0:
            smiles.append(cc[0].isomeric_smiles)
            inchikey.append(cc[0].inchikey)
        else:
            smiles.append(np.NAN)
            inchikey.append(np.NAN)
    else:
        smiles.append(row['SMILES'])
        mol = Chem.MolFromSmiles(row['SMILES'])
        inchikey.append(Chem.MolToInchiKey(mol))
    # break


100%|██████████| 640/640 [00:04<00:00, 158.97it/s]


In [9]:
df['smiles_fetched']=smiles
df['inchikey_feched']=inchikey

In [11]:
df_fetched = df[['Common name', 'smiles_fetched','RT (-) TTOF','Mix filename (pos)','Mix filename (neg)', 'inchikey_feched']]

In [12]:
from toolsets.std_list_prep import neutrilize_salt_df, neutrilize_salt, std_list_cleanup

In [92]:
df_cleaned =neutrilize_salt_df(df_fetched)

In [75]:
from molmass import Formula

In [93]:
labels = []
for index, row in df_cleaned.iterrows():
    if Formula(row['original_formula']).formula !=Formula(row['uncharged_formula']).formula:
        labels.append('check')
    else:
        labels.append('pass')
df_cleaned['labels']=labels

In [103]:
df_cleaned['mix_pos']=

,common name,smiles_fetched,rt (-) ttof,mix filename (pos),mix filename (neg),inchikey_feched,original_smiles,original_formula,original_formal_charges,stripped_smiles,stripped_formula,stripped_formal_charges,uncharged_smiles,uncharged_formula,uncharged_formal_charges,monoisotopic_mass,labels
4,Meclocycline sulfosalicylate salt,OC(=O)c1cc(ccc1O)S(O)(=O)=O.[H][C@@]12[C@@H](O...,NaN,Polyphenols-pos-mix1,Polyphenols-neg-mix1,FYSVKUUNXYGFLA-CCHMMTNSSA-N,C=C1c2c(Cl)ccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(...,C29H27ClN2O14S,0,C=C1c2c(Cl)ccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(...,C22H21ClN2O8,0,C=C1c2c(Cl)ccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(...,C22H21ClN2O8,0,476.098643,check
9,DL-Adrenaline Hydrochloride,Cl.CNCC(O)c1ccc(O)c(O)c1,0.312,Polyphenols-pos-mix2,Polyphenols-neg-mix2,ATADHKWKHYVBTJ-UHFFFAOYSA-N,CNCC(O)c1ccc(O)c(O)c1.Cl,C9H14ClNO3,0,CNCC(O)c1ccc(O)c(O)c1,C9H13NO3,0,CNCC(O)c1ccc(O)c(O)c1,C9H13NO3,0,183.089543,check
13,Doxorubicin hydrochloride,Cl.COc1cccc2C(=O)c3c(O)c4C[C@](O)(C[C@H](O[C@H...,2.917,Polyphenols-pos-mix2,Polyphenols-neg-mix2,MWWSFMDVAYGXBV-RUELKSSGSA-N,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,C27H30ClNO11,0,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,C27H29NO11,0,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,C27H29NO11,0,543.174061,check
25,Vancomycin hydrochloride,Cl.CN[C@H](CC(C)C)C(=O)N[C@@H]1[C@H](O)c2ccc(O...,NaN,Polyphenols-pos-mix3,Polyphenols-neg-mix3,LCTORFDMHNKUSG-XTTLPDOESA-N,CN[C@H](CC(C)C)C(=O)N[C@H]1C(=O)N[C@@H](CC(N)=...,C66H76Cl3N9O24,0,CN[C@H](CC(C)C)C(=O)N[C@H]1C(=O)N[C@@H](CC(N)=...,C66H75Cl2N9O24,0,CN[C@H](CC(C)C)C(=O)N[C@H]1C(=O)N[C@@H](CC(N)=...,C66H75Cl2N9O24,0,1447.430200,check
34,Doxycycline (hyclate),O.Cl.Cl.CCO.C[C@@H]1[C@H]2[C@H](O)[C@H]3[C@H](...,NaN,Polyphenols-pos-mix4,Polyphenols-neg-mix4,UHHHTIKWXBRCLT-VDBOFHIQSA-N,CCO.C[C@H]1c2cccc(O)c2C(O)=C2C(=O)[C@]3(O)C(O)...,C46H58Cl2N4O18,0,CCO.C[C@H]1c2cccc(O)c2C(O)=C2C(=O)[C@]3(O)C(O)...,C46H54N4O17,0,CCO.C[C@H]1c2cccc(O)c2C(O)=C2C(=O)[C@]3(O)C(O)...,C46H54N4O17,0,934.348396,check
50,Danshensu sodium salt,[Na+].OC(Cc1ccc(O)c(O)c1)C([O-])=O,1.187,Polyphenols-pos-mix6,Polyphenols-neg-mix6,ZMMKVDBZTXUHFO-UHFFFAOYSA-M,O=C([O-])C(O)Cc1ccc(O)c(O)c1.[Na+],C9H9NaO5,0,O=C(O)C(O)Cc1ccc(O)c(O)c1,C9H10O5,0,O=C(O)C(O)Cc1ccc(O)c(O)c1,C9H10O5,0,198.052823,check
54,(+)-Catechin Hydrate,O.O[C@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1ccc(O)c(O)c1,NaN,Polyphenols-pos-mix6,Polyphenols-neg-mix6,OFUMQWOJBVNKLR-NQQJLSKUSA-N,O.Oc1cc(O)c2c(c1)O[C@H](c1ccc(O)c(O)c1)[C@@H](...,C15H16O7,0,Oc1cc(O)c2c(c1)O[C@H](c1ccc(O)c(O)c1)[C@@H](O)C2,C15H14O6,0,Oc1cc(O)c2c(c1)O[C@H](c1ccc(O)c(O)c1)[C@@H](O)C2,C15H14O6,0,290.079038,check
90,Daunorubicin hydrochloride,Cl.COc1cccc2C(=O)c3c(O)c4C[C@](O)(C[C@H](O[C@H...,NaN,Polyphenols-pos-mix10,Polyphenols-neg-mix10,GUGHGUXZJWAIAS-QQYBVWGSSA-N,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,C27H30ClNO10,0,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,C27H29NO10,0,COc1cccc2c1C(=O)c1c(O)c3c(c(O)c1C2=O)C[C@@](O)...,C27H29NO10,0,527.179146,check
94,Higenamine hydrochloride,Cl.Oc1ccc(CC2NCCc3cc(O)c(O)cc23)cc1,1.606,Polyphenols-pos-mix10,Polyphenols-neg-mix10,SWWQQSDRUYSMAR-UHFFFAOYSA-N,Cl.Oc1ccc(CC2NCCc3cc(O)c(O)cc32)cc1,C16H18ClNO3,0,Oc1ccc(CC2NCCc3cc(O)c(O)cc32)cc1,C16H17NO3,0,Oc1ccc(CC2NCCc3cc(O)c(O)cc32)cc1,C16H17NO3,0,271.120843,check
114,Quercetin Dihydrate,O.O.Oc1cc(O)c2c(c1)oc(-c1ccc(O)c(O)c1)c(O)c2=O,NaN,Polyphenols-pos-mix12,Polyphenols-neg-mix12,GMGIWEZSKCNYSW-UHFFFAOYSA-N,O.O.O=c1c(O)c(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12,C15H14O9,0,O=c1c(O)c(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12,C15H10O7,0,O=c1c(O)c(-c2ccc(O)c(O)c2)oc2cc(O)cc(O)c12,C15H10O7,0,302.042653,check


In [70]:
from rdkit.Chem.Draw import IPythonConsole

IPythonConsole.molSize = (800, 800)

In [79]:
from rdkit.Chem.rdMolDescriptors import CalcMolFormula
from rdkit.Chem.Descriptors import ExactMolWt

In [105]:
std_list_pos = { 'name': df_cleaned['common name'],
             # 'abb':std_list_nt['abbreviated'],
             # 'inchikey': std_list_nt['uncharged_inchikey'],
             'mix':df_cleaned['mix filename (pos)'],
            'rt_reference':df_cleaned['rt (-) ttof'],
             'smiles':df_cleaned['uncharged_smiles'],
             'formula':df_cleaned['uncharged_formula'],
             # 'mono_mass':df_cleaned['monoisotopic_mass'],
             'formal_charges':df_cleaned['uncharged_formal_charges'],
            'label':df_cleaned['labels']
             # 'rt_plus':std_list_nt['rt_plus'],
             # 'rt_minus':std_list_nt['rt_minus']
             }
std_list_pos = pd.DataFrame(std_list_pos)
std_list_neg = { 'name': df_cleaned['common name'],
                 # 'abb':std_list_nt['abbreviated'],
                 # 'inchikey': std_list_nt['uncharged_inchikey'],
                 'mix':df_cleaned['mix filename (neg)'],
                 'rt_reference':df_cleaned['rt (-) ttof'],
                 'smiles':df_cleaned['uncharged_smiles'],
                 'formula':df_cleaned['uncharged_formula'],
                 # 'mono_mass':df_cleaned['monoisotopic_mass'],
                 'formal_charges':df_cleaned['uncharged_formal_charges'],
                 'label':df_cleaned['labels'],
                 # 'rt_plus':std_list_nt['rt_plus'],
                 # 'rt_minus':std_list_nt['rt_minus']
                 }
std_list_neg = pd.DataFrame(std_list_neg)

In [106]:
std_list_pos

,name,mix,rt_reference,smiles,formula,formal_charges,label
0,Theaflavin-3-gallate,Polyphenols-pos-mix1,2.958,O=C(O[C@@H]1Cc2c(O)cc(O)cc2O[C@@H]1c1cc(O)c(=O...,C36H28O16,0,pass
1,Daurisoline,Polyphenols-pos-mix1,2.311,COc1cc2c(cc1O)[C@@H](Cc1ccc(O)c(Oc3ccc(C[C@@H]...,C37H42N2O6,0,pass
2,Ellagic acid,Polyphenols-pos-mix1,2.227,O=c1oc2c(O)c(O)cc3c(=O)oc4c(O)c(O)cc1c4c23,C14H6O8,0,pass
3,Methyl-Hesperidin,Polyphenols-pos-mix1,2.882,COc1ccc([C@@H]2CC(=O)c3c(O)cc(O[C@@H]4O[C@H](C...,C29H36O15,0,pass
4,Meclocycline sulfosalicylate salt,Polyphenols-pos-mix1,NaN,C=C1c2c(Cl)ccc(O)c2C(=O)C2=C(O)[C@]3(O)C(=O)C(...,C22H21ClN2O8,0,check
...,...,...,...,...,...,...,...
635,Apigenin,Polyphenols-pos-mix66,3.266,O=c1cc(-c2ccc(O)cc2)oc2cc(O)cc(O)c12,C15H10O5,0,pass
636,Laetanine,Polyphenols-pos-mix66,1.959,COc1cc2c(cc1O)-c1c(OC)c(O)cc3c1[C@H](C2)NCC3,C18H19NO4,0,pass
637,Apigenin-7-O-(2G-rhamnosyl)gentiobioside,Polyphenols-pos-mix66,2.285,CC1OC(OC2C(Oc3cc(O)c4c(=O)cc(-c5ccc(O)cc5)oc4c...,C33H40O19,0,pass
638,Cirsimaritin,Polyphenols-pos-mix66,3.772,COc1cc2oc(-c3ccc(O)cc3)cc(=O)c2c(O)c1OC,C17H14O6,0,pass


In [113]:
std_list_pos.to_csv(os.path.join(master_dir, 'std_list_pos.csv'), index = False)
std_list_neg.to_csv(os.path.join(master_dir, 'std_list_neg.csv'), index = False)